In [1]:
import os
import time
import pickle
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import polars as pl
import scipy.stats
import pywt
from sklearn.model_selection import KFold, train_test_split
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm

class Config:
    DATA_PATH = '/kaggle/input/ariel-data-challenge-2025/'
    PREPROCESSED_PATH = '/kaggle/input/ariel-data-challenge-2025-af-npy/'
    OUTPUT_PATH = '/kaggle/working/'
    TRAIN_LABELS_PATH = os.path.join(DATA_PATH, 'train.csv')
    TRAIN_STAR_INFO_PATH = os.path.join(DATA_PATH, 'train_star_info.csv')
    TEST_STAR_INFO_PATH = os.path.join(DATA_PATH, 'test_star_info.csv')
    SAMPLE_SUBMISSION_PATH = os.path.join(DATA_PATH, 'sample_submission.csv')
    WAVELENGTHS_PATH = os.path.join(DATA_PATH, 'wavelengths.csv')
    VALIDATION_FOLDS = 5
    RANDOM_STATE = 42
    QUANTILES = [0.05, 0.50, 0.95]
    XGB_PARAMS = {
        'n_estimators': 400,
        'learning_rate': 0.04,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.7,
        'random_state': RANDOM_STATE,
        'tree_method': 'hist',
        'verbosity': 0
    }
    LGB_PARAMS = {
        'learning_rate': 0.03,
        'num_leaves': 64,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 1,
        'seed': RANDOM_STATE,
        'n_estimators': 1000,
        'verbosity': -1
    }

config = Config()

def f_read_and_preprocess(dataset, planet_ids):
    f_raw = np.full((len(planet_ids), 67500), np.nan, dtype=np.float32)
    print(f"Processing FGS1 data for {len(planet_ids)} planets...")
    for i, pid in enumerate(tqdm(planet_ids, desc="Loading FGS1 signals")):
        df = pl.read_parquet(f"{config.DATA_PATH}/{dataset}/{int(pid)}/FGS1_signal_0.parquet")
        mean_signal = df.cast(pl.Int32).sum_horizontal().to_numpy()/1024
        f_raw[i] = mean_signal[1::2] - mean_signal[0::2]
    return f_raw


def a_read_and_preprocess(dataset, planet_ids):
    a_raw = np.full((len(planet_ids), 5625), np.nan, dtype=np.float32)
    print(f"Processing AIRS data for {len(planet_ids)} planets...")
    for i, pid in enumerate(tqdm(planet_ids, desc="Loading AIRS signals")):
        df = pl.read_parquet(f"{config.DATA_PATH}/{dataset}/{int(pid)}/AIRS-CH0_signal_0.parquet")
        mean_signal = df.cast(pl.Int32).sum_horizontal().to_numpy()/(32*356)
        a_raw[i] = mean_signal[1::2] - mean_signal[0::2]
    return a_raw


def make_features(f_raw, a_raw, star_info):
    print("Generating features...")
    
    # FGS1 features 
    fgs_pre = f_raw[:, :20500]; fgs_post = f_raw[:, 47000:]
    fgs_unmean = (fgs_pre.mean(1)+fgs_post.mean(1))/2
    fgs_unstd = (fgs_pre.std(1)+fgs_post.std(1))/2
    transit = f_raw[:,23500:44000]
    feats = {}
    
    # FGS1 slice features
    for i in tqdm(range(5), desc="FGS1 slice features", leave=False):
        m = transit[:,i*4100:(i+1)*4100].mean(1)
        feats[f'fgs_slice_{i+1}'] = (fgs_unmean - m)/fgs_unmean
    
    feats['fgs_transit_std'] = transit.std(1)
    feats['fgs_transit_skew'] = scipy.stats.skew(transit, axis=1)
    
    # FFT features
    fft = np.fft.fft(transit, axis=1)
    for i in tqdm(range(1,6), desc="FGS1 FFT features", leave=False): 
        feats[f'fgs_fft_{i}'] = np.abs(fft[:,i])
    
    # Wavelet features
    for lvl in tqdm(range(1,4), desc="FGS1 wavelet features", leave=False):
        c = pywt.wavedec(transit, 'db4', level=lvl, axis=1)[0]
        feats[f'fgs_wav_std_{lvl}'] = np.std(c,1)

    # AIRS features: central window
    a_tr = a_raw[:,500:4500]
    feats['airs_std'] = a_tr.std(1)
    feats['airs_skew'] = scipy.stats.skew(a_tr,axis=1)
    
    # AIRS FFT features
    fft_a = np.fft.fft(a_tr, axis=1)
    for i in tqdm(range(1,6), desc="AIRS FFT features", leave=False): 
        feats[f'airs_fft_{i}'] = np.abs(fft_a[:,i])
    
    # AIRS wavelet features
    for lvl in tqdm(range(1,4), desc="AIRS wavelet features", leave=False):
        c = pywt.wavedec(a_tr, 'db4', level=lvl, axis=1)[0]
        feats[f'airs_wav_std_{lvl}'] = np.std(c,1)

    # Metadata & interactions
    meta = star_info.fillna(star_info.median())
    meta['Mp_P'] = meta['Mp']/ (meta['P']+1e-6)
    meta['logRs_logTs'] = np.log1p(meta['Rs'])*np.log1p(meta['Ts'])

    X = pd.DataFrame(feats, index=star_info.index)
    X = pd.concat([X, meta], axis=1)
    X.fillna(0, inplace=True)
    print(f"Generated {X.shape[1]} features for {X.shape[0]} samples")
    return X

def official_score(y_true, y_pred, sigma, mu0, sigma0):
    y_true,y_pred,sigma = map(np.array, (y_true,y_pred,sigma))
    sigma = np.clip(sigma,1e-15,None)
    base = scipy.stats.norm.logpdf(y_true,loc=y_true,scale=sigma0)
    glp = scipy.stats.norm.logpdf(y_true,loc=y_pred,scale=sigma)
    gm = scipy.stats.norm.logpdf(y_true,loc=mu0,scale=sigma0)
    return ((glp-gm)/(base-gm+1e-9)).mean()

def train_and_calibrate():
    print("Loading training data...")
    labels = pd.read_csv(config.TRAIN_LABELS_PATH, index_col='planet_id')
    stars = pd.read_csv(config.TRAIN_STAR_INFO_PATH, index_col='planet_id').loc[labels.index]
    
    print("Loading preprocessed arrays...")
    f_raw = np.load(config.PREPROCESSED_PATH+'f_raw_train.npy')
    a_raw = np.load(config.PREPROCESSED_PATH+'a_raw_train.npy')
    
    X = make_features(f_raw, a_raw, stars)
    y = labels.values
    mu0, sigma0 = y.mean(), y.std()

    # OOF containers
    oof_med = np.zeros_like(y)
    oof_sig = np.zeros_like(y)
    raw_sig_vals = []

    print(f"Starting {config.VALIDATION_FOLDS}-fold cross-validation...")
    kf = KFold(n_splits=config.VALIDATION_FOLDS, shuffle=True, random_state=config.RANDOM_STATE)
    
    for fold, (tr, val) in enumerate(tqdm(kf.split(X), total=config.VALIDATION_FOLDS, desc="CV Folds")):
        print(f"\n--- Fold {fold + 1}/{config.VALIDATION_FOLDS} ---")
        X_tr, X_val = X.iloc[tr], X.iloc[val]
        y_tr, y_val = y[tr], y[val]
        
        # train median with stacking: XGB + LGB + Ridge
        preds = []
        models = ['XGBoost', 'LightGBM']
        
        for idx, mdl in enumerate(tqdm(['xgb','lgb'], desc="Training models", leave=False)):
            if mdl=='xgb':
                print(f"  Training {models[idx]}...")
                m = xgb.XGBRegressor(**config.XGB_PARAMS, objective='reg:squarederror')
                m.fit(X_tr, y_tr)
            else:
                print(f"  Training {models[idx]}...")
                m = MultiOutputRegressor(lgb.LGBMRegressor(**config.LGB_PARAMS, objective='regression'))
                m.fit(X_tr, y_tr)
            preds.append(m.predict(X_val))
        
        # stack
        print("  Stacking predictions...")
        stack_tr = np.stack(preds,2).mean(2)
        ridge = Ridge().fit(stack_tr, y_val)
        med = ridge.predict(stack_tr)
        oof_med[val] = med
        
        # quantile raw
        print("  Training quantile regressors...")
        lower = MultiOutputRegressor(lgb.LGBMRegressor(**config.LGB_PARAMS, objective='quantile', alpha=0.05)).fit(X_tr,y_tr).predict(X_val)
        upper = MultiOutputRegressor(lgb.LGBMRegressor(**config.LGB_PARAMS, objective='quantile', alpha=0.95)).fit(X_tr,y_tr).predict(X_val)
        raw = (upper-lower)/3.29
        oof_sig[val] = raw
        raw_sig_vals.append((raw.flatten(), np.abs(y_val-med).flatten()))

    # isotonic calibration
    print("\nPerforming isotonic calibration...")
    raw_all = np.concatenate([r for r,_ in raw_sig_vals])
    true_err = np.concatenate([e for _,e in raw_sig_vals])
    ir = IsotonicRegression(out_of_bounds='clip').fit(raw_all, true_err)
    
    # save calibration
    print("Saving calibrator and features...")
    with open(config.OUTPUT_PATH+'calibrator.pkl','wb') as f: pickle.dump(ir,f)
    # save features
    with open(config.OUTPUT_PATH+'features.pkl','wb') as f: pickle.dump(X.columns.tolist(),f)
    
    # return OOF score
    sig_cal = ir.predict(oof_sig.flatten()).reshape(oof_sig.shape)
    print(f"\nOOF official score: {official_score(y,oof_med,sig_cal,mu0,sigma0):.6f}")

def make_submission():
    print("Creating submission...")
    # load
    print("Loading submission data...")
    sample = pd.read_csv(config.SAMPLE_SUBMISSION_PATH, index_col='planet_id')
    stars = pd.read_csv(config.TEST_STAR_INFO_PATH, index_col='planet_id')
    waves = pd.read_csv(config.WAVELENGTHS_PATH)
    
    print("Loading saved models...")
    cols = pickle.load(open(config.OUTPUT_PATH+'features.pkl','rb'))
    ir = pickle.load(open(config.OUTPUT_PATH+'calibrator.pkl','rb'))

    # data
    f_raw = f_read_and_preprocess('test', stars.index)
    a_raw = a_read_and_preprocess('test', stars.index)
    X_test = make_features(f_raw,a_raw,stars)[cols]

    # median ensemble
    print("Training final models on full data...")
    print("  Training XGBoost...")
    pred_xgb = xgb.XGBRegressor(**config.XGB_PARAMS).fit(X_test, np.zeros((0, waves.shape[1]))).predict(X_test)
    print("  Training LightGBM...")
    pred_lgb = MultiOutputRegressor(lgb.LGBMRegressor(**config.LGB_PARAMS, objective='regression')).fit(X_test, np.zeros((0,waves.shape[1]))).predict(X_test)
    print("  Stacking predictions...")
    med = Ridge().fit(np.stack([pred_xgb,pred_lgb],2).mean(2), np.zeros((0,waves.shape[1]))).predict(np.stack([pred_xgb,pred_lgb],2).mean(2))
    
    # quantiles
    print("  Training quantile regressors...")
    lower = MultiOutputRegressor(lgb.LGBMRegressor(**config.LGB_PARAMS, objective='quantile', alpha=0.05)).fit(X_test,np.zeros((0,waves.shape[1]))).predict(X_test)
    upper = MultiOutputRegressor(lgb.LGBMRegressor(**config.LGB_PARAMS, objective='quantile', alpha=0.95)).fit(X_test,np.zeros((0,waves.shape[1]))).predict(X_test)
    raw = (upper-lower)/3.29
    
    print("  Applying calibration...")
    sig = ir.predict(raw.flatten()).reshape(raw.shape)

    # build df
    print("Building submission DataFrame...")
    df_med = pd.DataFrame(np.clip(med,0,None), index=sample.index, columns=waves.columns)
    df_sig = pd.DataFrame(sig, index=sample.index, columns=[f'sigma_{i+1}' for i in range(waves.shape[1])])
    sub = pd.concat([df_med, df_sig], axis=1)
    
    print("Saving submission...")
    sub.to_csv('submission.csv')
    print('✓ submission.csv created successfully!')

# ==============================================================================
# ENTRY POINT
# ==============================================================================
#if __name__=='__main__':
#    mode = os.getenv('MODE','train')  # set MODE=train or MODE=submit
#    print(f"Running in {mode} mode...")
#    if mode=='train': 
#        train_and_calibrate()
#    else: 
#        make_submission()

In [2]:
def make_submission():
    print("Creating submission...")
    
    labels = pd.read_csv(config.TRAIN_LABELS_PATH, index_col='planet_id')
    stars_train = pd.read_csv(config.TRAIN_STAR_INFO_PATH, index_col='planet_id').loc[labels.index]
    f_train = np.load(config.PREPROCESSED_PATH+'f_raw_train.npy')
    a_train = np.load(config.PREPROCESSED_PATH+'a_raw_train.npy')
    X_train = make_features(f_train, a_train, stars_train)
    y_train = labels.values
    print(f"Final training on {X_train.shape[0]} samples x {X_train.shape[1]} features.")

    sample = pd.read_csv(config.SAMPLE_SUBMISSION_PATH, index_col='planet_id')
    stars_test = pd.read_csv(config.TEST_STAR_INFO_PATH, index_col='planet_id')
    waves = pd.read_csv(config.WAVELENGTHS_PATH)

    f_test = f_read_and_preprocess('test', stars_test.index)
    a_test = a_read_and_preprocess('test', stars_test.index)
    X_test = make_features(f_test, a_test, stars_test)

    print("  Training XGBoost on full data...")
    xgb_full = xgb.XGBRegressor(**config.XGB_PARAMS, objective='reg:squarederror')
    xgb_full.fit(X_train, y_train)

    print("  Training LightGBM on full data...")
    lgb_full = MultiOutputRegressor(
        lgb.LGBMRegressor(**config.LGB_PARAMS, objective='regression')
    )
    lgb_full.fit(X_train, y_train)

    print("  Forming median ensemble...")
    pred_x = xgb_full.predict(X_test)
    pred_l = lgb_full.predict(X_test)
    med = Ridge().fit(
        np.stack([xgb_full.predict(X_train), lgb_full.predict(X_train)], axis=2).mean(2),
        y_train
    ).predict(np.stack([pred_x, pred_l], axis=2).mean(2))

    print("  Training quantile regressors...")
    lower_mdl = MultiOutputRegressor(
        lgb.LGBMRegressor(**config.LGB_PARAMS, objective='quantile', alpha=0.05)
    ).fit(X_train, y_train)
    upper_mdl = MultiOutputRegressor(
        lgb.LGBMRegressor(**config.LGB_PARAMS, objective='quantile', alpha=0.95)
    ).fit(X_train, y_train)
    lower = lower_mdl.predict(X_test)
    upper = upper_mdl.predict(X_test)
    raw_sigma = (upper - lower) / 3.29

    print(raw_sigma)

    #ir = pickle.load(open("/kaggle/input/neurips-pkls/calibrator.pkl",'rb'))
    sigma = raw_sigma.flatten().reshape(raw_sigma.shape)

    df_med = pd.DataFrame(np.clip(med, 0, None),
                          index=sample.index, columns=waves.columns)
    df_sig = pd.DataFrame(sigma,
                          index=sample.index,
                          columns=[f'sigma_{i+1}' for i in range(waves.shape[1])])
    #df_sig = pd.DataFrame(raw_sigma,
                          #index=sample.index,
                          #columns=[f'sigma_{i+1}' for i in range(waves.shape[1])])
    submission = pd.concat([df_med, df_sig], axis=1)
    submission.to_csv('submission.csv')
    print("submission created successfully!")

In [3]:
make_submission()

Creating submission...
Generating features...


Generated 35 features for 1100 samples
Final training on 1100 samples x 35 features.
Processing FGS1 data for 1 planets...


Loading FGS1 signals: 100%|██████████| 1/1 [00:01<00:00,  1.04s/it]


Processing AIRS data for 1 planets...


Loading AIRS signals: 100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


Generating features...


Generated 35 features for 1 samples
  Training XGBoost on full data...
  Training LightGBM on full data...
  Forming median ensemble...
  Training quantile regressors...
[[ 1.62281851e-05  5.45407689e-04  6.15054629e-05  4.09784934e-04
   2.19479510e-04  4.36732769e-04  4.60180664e-05  4.59631481e-04
   3.84757662e-04  5.55488075e-04  3.46087838e-04  2.18335866e-04
   6.15342206e-04  3.22037099e-04  2.73908537e-04  4.41517333e-04
   3.73951051e-04  6.22037420e-04  6.05126574e-04  4.39042981e-04
   4.12717429e-04  5.29294003e-04  3.65585705e-04  5.49047041e-04
   5.81348449e-04  5.44117142e-04  3.76045376e-04  5.54531321e-04
   7.48097231e-04  4.55351715e-04  3.58565403e-04  7.12598537e-04
   4.97126259e-04  4.19664699e-04  3.76780441e-04  3.44794052e-04
   2.55964322e-04  3.31586362e-04  5.95279003e-04  5.27430259e-04
   3.56778800e-04  3.40738771e-04  3.82365743e-04  4.75274765e-04
   3.76874316e-04  5.18045579e-04  3.30334786e-04  5.33421467e-04
   5.27068551e-04  4.66945626e-04  6.9